# Efficient Processing and Analysis of Large JSON Datasets with DuckDB

_Author: Till Bey_

This notebook demonstrates how to efficiently process, transform, and analyze large JSON datasets using DuckDB. The workflow includes converting JSON files into Parquet format, extracting schemas, querying specific data, and performing batch processing for scalability. The steps are outlined as follows:

1. **Processing JSON Files with DuckDB**: Transform large, nested JSON datasets into compressed Parquet files for efficient storage and querying.
2. **Extracting Schema from Parquet Files**: Retrieve and save the schema of Parquet files for documentation or further use.
3. **Querying Specific Data**: Perform targeted queries, such as retrieving titles with specific reference counts or counting records by publication year.
4. **Batch Processing of JSON Files**: Split and process large datasets in manageable batches, converting them into Parquet format.
5. **Converting Large JSON Files**: Handle large JSON files and convert them into a single Parquet file for streamlined analysis.
6. **Counting and Visualizing Records**: Analyze the distribution of records over time and visualize the results.
7. **Metadata Extraction**: Extract metadata from Parquet files for additional insights.

This notebook provides a comprehensive guide for handling large-scale JSON data, leveraging DuckDB's capabilities for efficient data processing and analysis.


In [1]:
import duckdb
import os
import math

### Processing JSON Files with DuckDB

This operation is useful for transforming large, nested JSON datasets into a more efficient and queryable format like Parquet.his code uses DuckDB to process JSON files and convert them into a compressed Parquet file. Here's what it does step by step:

1. **`read_json`**: Reads all JSON files from the directory `crossref_202404_sample/*.json.gz`. The `sample_size=-1` parameter indicates that all data should be read, and `union_by_name=true` ensures that fields are aligned by name across the files.

2. **`unnest(items, max_depth:=2)`**: Flattens nested JSON structures up to a depth of 2, making the data easier to work with.

3. **`COPY`**: Writes the processed data into a Parquet file named `crossref_2024_04_sample.gz.parquet`. The file is compressed using Gzip to save storage space.

In [3]:
duckdb.sql('''
    COPY (
        SELECT unnest(items, max_depth:=2)
        FROM (
            SELECT *
            FROM read_json('crossref_202404_sample/*.json.gz', sample_size=-1, union_by_name=true)
        )
    )
    TO 'crossref_2024_04_sample.gz.parquet' (FORMAT parquet, COMPRESSION gzip)
''')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Extracting Schema from Parquet File

This code uses DuckDB to extract the schema of a Parquet file and save it to a SQL file. Here's what it does:

1. **`DESCRIBE`**: Retrieves the schema of the Parquet file `crossref_2024_04_sample.snappy.parquet`, including details about its columns and data types.

2. **`COPY`**: Writes the extracted schema into a file named `schema.sql` for documentation or further use.

In [17]:
duckdb.sql('''
    COPY (
        SELECT *
        FROM (DESCRIBE 'crossref_2024_04_sample.snappy.parquet')
    )
    TO 'schema.sql'
''')

### Querying Titles with Specific Reference Count

This query retrieves the titles of records from the Parquet file `crossref_2024_04_sample.snappy.parquet` where the `"reference-count"` is equal to 14. This can be useful for analyzing or extracting specific entries based on their reference count.

In [6]:
duckdb.sql('''
    SELECT title
    FROM 'crossref_2024_04_sample.snappy.parquet'
    WHERE "reference-count" = 14
    
''')

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                          title                                                                                                          │
│                                                                                                        varchar[]                                                                                                        │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ [Stereoselective Syntheses of the Three Isomers of Ethylene Glycol Bis(tropane-3-carboxylate)]                        

### Counting Records by Publication Year

This query counts the number of records in the Parquet file `crossref_2024_04_sample.snappy.parquet` grouped by the publication year. It extracts the year from the `"date-parts"` field, groups the records by this year, and orders the results in descending order of publication year. This is useful for analyzing the distribution of records over time.

In [ ]:
duckdb.sql('''
    SELECT count(*), published."date-parts"[1][1] as publication_year
    FROM 'crossref_2024_04_sample.snappy.parquet'
    GROUP BY publication_year
    ORDER BY publication_year DESC
''')

CPU times: user 3.48 ms, sys: 2.36 ms, total: 5.84 ms
Wall time: 5.52 ms


┌──────────────┬──────────────────┐
│ count_star() │ publication_year │
│    int64     │      int64       │
├──────────────┼──────────────────┤
│            1 │             2034 │
│            7 │             2025 │
│         6450 │             2024 │
│          836 │             2023 │
│          159 │             2022 │
│           98 │             2021 │
│           60 │             2020 │
│           60 │             2019 │
│           25 │             2018 │
│           27 │             2017 │
│            · │               ·  │
│            · │               ·  │
│            · │               ·  │
│            1 │             1893 │
│            1 │             1886 │
│            2 │             1882 │
│            1 │             1881 │
│            1 │             1858 │
│            2 │             1851 │
│            1 │             1848 │
│            1 │             1832 │
│            1 │             1811 │
│          652 │             NULL │
├──────────────┴────────────

## Full dataset, bit by bit

This code generates a list of file paths for JSON files in the directory `crossref_202404`. Here's how it works:

1. **`os.listdir('crossref_202404')`**: Lists all files in the directory `crossref_202404`.

2. **`'0'*(13-len(f)) + f`**: Pads each filename with leading zeros to ensure a uniform length of 13 characters.

3. **`sorted(...)`**: Sorts the padded filenames in ascending order.

4. **`'0.json.gz' if f == '00000.json.gz' else f.strip('0')`**: Replaces the filename `00000.json.gz` with `0.json.gz` and removes leading zeros from other filenames.

5. **`os.path.join('crossref_202404', ...)`**: Joins the directory path with the processed filenames to create full file paths.

The resulting list, `filenames`, contains the full paths of the JSON files in the directory, sorted and formatted as described.

In [2]:
filenames = [os.path.join('crossref_202404', '0.json.gz' if f == '00000.json.gz' else f.strip('0'))
             for f in sorted('0'*(13-len(f)) + f for f in os.listdir('crossref_202404'))]
filenames

['crossref_202404/0.json.gz',
 'crossref_202404/1.json.gz',
 'crossref_202404/2.json.gz',
 'crossref_202404/3.json.gz',
 'crossref_202404/4.json.gz',
 'crossref_202404/5.json.gz',
 'crossref_202404/6.json.gz',
 'crossref_202404/7.json.gz',
 'crossref_202404/8.json.gz',
 'crossref_202404/9.json.gz',
 'crossref_202404/10.json.gz',
 'crossref_202404/11.json.gz',
 'crossref_202404/12.json.gz',
 'crossref_202404/13.json.gz',
 'crossref_202404/14.json.gz',
 'crossref_202404/15.json.gz',
 'crossref_202404/16.json.gz',
 'crossref_202404/17.json.gz',
 'crossref_202404/18.json.gz',
 'crossref_202404/19.json.gz',
 'crossref_202404/20.json.gz',
 'crossref_202404/21.json.gz',
 'crossref_202404/22.json.gz',
 'crossref_202404/23.json.gz',
 'crossref_202404/24.json.gz',
 'crossref_202404/25.json.gz',
 'crossref_202404/26.json.gz',
 'crossref_202404/27.json.gz',
 'crossref_202404/28.json.gz',
 'crossref_202404/29.json.gz',
 'crossref_202404/30.json.gz',
 'crossref_202404/31.json.gz',
 'crossref_202404/

### Splitting and Processing JSON Files in Batches

This code processes a large number of JSON files in batches of 500, converting them into Parquet format for efficient storage and querying. Here's how it works:

1. **`math.ceil(len(filenames)/500)`**: Calculates the number of batches required by dividing the total number of files by 500 and rounding up.

2. **Progress Bar**: Displays a progress bar in the terminal to indicate the processing status of the batches.

3. **Batch Processing**:
    - For each batch, selects up to 500 filenames from the `filenames` list.
    - Reads the JSON files using DuckDB's `read_json` function with parameters to handle large files and align fields by name.
    - Flattens nested JSON structures up to a depth of 2 using `unnest`.
    - Writes the processed data into a Parquet file named `crossref_202404_{i}.parquet` in the `crossref_202404_parquet` directory.

4. **Dynamic Progress Update**: Updates the progress bar dynamically after processing each batch, displaying the last processed filename.

In [3]:
n = math.ceil(len(filenames)/500)
print('\r\u2595' + n*'\u2581' + '\u258F', end='')

for i in range(n):
    selected_filenames = filenames[i*500:(i+1)*500]
    duckdb.sql(f'''
        PRAGMA disable_progress_bar;
        COPY (
                SELECT unnest(items, max_depth:=2)
                FROM (
                    SELECT *
                    FROM read_json({selected_filenames}, sample_size=100000,
                                    union_by_name=true, maximum_object_size:=100000000)
                )
            )
            TO 'crossref_202404_parquet/crossref_202404_{i}.parquet'
    ''')
    print('\r\u2595' + (i+1)*'\u2587' + (n-i)*'\u2581' + '\u258F ' + selected_filenames[-1], end='')


▕▇▇▇▇▇▇▇▇▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▏ crossref_202404/4499.json.gz

InvalidInputException: Invalid Input Error: "maximum_object_size" of 100000000 bytes exceeded while reading file "crossref_202404/4736.json.gz" (>108807049 bytes).
 Try increasing "maximum_object_size".

### Converting Large JSON Files to Parquet Format

This code processes large JSON files in the directory `crossref_202404_large_sample` and converts them into a single Parquet file named `crossref_202404_1.parquet`. Here's how it works:

1. **`read_json`**: Reads JSON files from the specified directory with a sample size of 100,000 records. The `union_by_name=true` parameter ensures that fields are aligned by name across the files, and `maximum_object_size:=100000000` handles large objects efficiently.

2. **`unnest(items, max_depth:=2)`**: Flattens nested JSON structures up to a depth of 2, simplifying the data for further processing.

3. **`COPY`**: Writes the processed data into a Parquet file named `crossref_202404_1.parquet`. Parquet is a columnar storage format that is optimized for analytical queries and efficient storage.

In [2]:
duckdb.sql('''
    COPY (
        SELECT unnest(items, max_depth:=2)
        FROM (
            SELECT *
            FROM read_json('crossref_202404_large_sample/*.json.gz', sample_size=100000, union_by_name=true, maximum_object_size:=100000000)
        )
    )
    TO 'crossref_202404_1.parquet'
''')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Counting Total Records in Parquet Files

This query calculates the total number of records across all Parquet files in the directory `crossref_202404_parquet`. It provides a quick overview of the dataset size, which is useful for understanding the scale of the data being analyzed.

In [19]:
duckdb.sql('''
    SELECT count(*)
    FROM 'crossref_202404_parquet/*.parquet'
''')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     22500000 │
└──────────────┘

### Counting and Visualizing Records by Publication Year

This query retrieves the publication year (`pubyear`) and the count of records for each year from the Parquet files in the directory `crossref_202404_parquet`. The results are grouped by publication year and ordered in descending order. The query can be extended to visualize the data as a bar chart by converting the result to a DataFrame and plotting it. This is useful for analyzing the distribution of records over time. 

In [20]:
duckdb.sql('''
    SELECT published."date-parts"[1][1] as pubyear, count(*)
    FROM 'crossref_202404_parquet/*.parquet'
    GROUP BY pubyear
    ORDER BY pubyear DESC
''')#.to_df().plot.bar(x='pubyear', y='count_star()')

┌─────────┬──────────────┐
│ pubyear │ count_star() │
│  int64  │    int64     │
├─────────┼──────────────┤
│    2200 │            1 │
│    2150 │            1 │
│    2121 │            1 │
│    2108 │           18 │
│    2107 │            1 │
│    2099 │            7 │
│    2096 │            1 │
│    2070 │            1 │
│    2058 │            7 │
│    2054 │            1 │
│      ·  │            · │
│      ·  │            · │
│      ·  │            · │
│    1693 │            1 │
│    1688 │            2 │
│    1687 │            4 │
│    1686 │            1 │
│    1685 │            4 │
│    1684 │            1 │
│    1651 │            2 │
│    1600 │            3 │
│    1400 │            3 │
│    NULL │      1163440 │
├─────────┴──────────────┤
│  358 rows (20 shown)   │
└────────────────────────┘

In [6]:
duckdb.sql('''
    SELECT count()
    FROM 'crossref_202404_parquet/*.parquet'
''')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     22500000 │
└──────────────┘

In [10]:
duckdb.sql('''
    SELECT *
    FROM (
        SELECT unnest(author) as author
        FROM 'crossref_202404_parquet/*.parquet'
    )
    WHERE author.given = 'Till D.' AND author.family = 'Bey'
''')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                                                    author                                                                                                                                                    │
│ struct(given varchar, "family" varchar, "sequence" varchar, affiliation struct("name" varchar, place varchar[], id struct(id varchar, "id-type" varchar, "asserted-by" varchar)[], department varchar[], acronym varchar[])[], suffix varchar, "name" varchar, orcid varchar, "authenticated-orcid" boolean) │
├────────────────────────────────────────────────────────────────────────────────────

In [21]:
duckdb.sql('''
    SELECT count(*), published."date-parts"[1][1] as pubyear
    FROM 'crossref_202404_parquet/*.parquet'
    WHERE title[1] LIKE '%covid%'
    GROUP BY pubyear
    ORDER BY pubyear
''')

┌──────────────┬─────────┐
│ count_star() │ pubyear │
│    int64     │  int64  │
├──────────────┼─────────┤
│            1 │    1998 │
│            1 │    2009 │
│            1 │    2014 │
│           57 │    2020 │
│          222 │    2021 │
│         1285 │    2022 │
│         2344 │    2023 │
│          406 │    2024 │
│           80 │    NULL │
└──────────────┴─────────┘

In [10]:
# %%time
duckdb.sql('''
    SELECT count(*), published."date-parts"[1][1] as pubyear
    FROM 'crossref_202404_parquet/*.parquet'
    WHERE title[1] ILIKE '%COVID%'
    GROUP BY pubyear
    ORDER BY pubyear
''')

┌──────────────┬─────────┐
│ count_star() │ pubyear │
│    int64     │  int64  │
├──────────────┼─────────┤
│            1 │    1920 │
│            3 │    1969 │
│           14 │    1970 │
│            3 │    1975 │
│            1 │    1996 │
│            6 │    1998 │
│            1 │    1999 │
│            2 │    2000 │
│            1 │    2001 │
│            2 │    2005 │
│            · │      ·  │
│            · │      ·  │
│            · │      ·  │
│           35 │    2019 │
│         2391 │    2020 │
│         7786 │    2021 │
│        62944 │    2022 │
│       132900 │    2023 │
│        28883 │    2024 │
│           30 │    2025 │
│            1 │    2030 │
│            1 │    2099 │
│         2142 │    NULL │
├──────────────┴─────────┤
│   29 rows (20 shown)   │
└────────────────────────┘

In [29]:
duckdb.sql('''
    SELECT *
    FROM parquet_metadata('crossref_202404_large_sample.parquet');
''')

┌──────────────────────────────────────┬──────────────┬────────────────────┬───────────────────────┬─────────────────┬───────────┬─────────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────┬──────────────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────┬─────────────┬────────────────┬───────────────────┬────────────────────────┬──────────────────┬───────────────────────┬─────────────────────────┬────────────────────┬─────────────────────┬─────────────────────┐
│              file_name               │ row_group_id │ row_group_num_rows │ row_group_num_columns │ row_group_bytes │ column_id │ file_offset │ num_values │                              

Files,json.gzip,snappy.parquet,records
0-499,4.24,5.26,2500000
500-999,3.44,4.07,2500000
